In [ ]:
#资产负债表分析思路，首先根据财务的一些关联对数据进行分块分析，相关性，线性回归模型拟合
##结果：
#财务公式                                                             	线性回归系数	                                                                       评分	选取做回归的样本占总样本比例
#流动资产合计=货币资金+应收账款+其它应收款+存货 	[1.48661497 1.06789572 1.35391323 0.94856807] b:23869542.65	0.9979  	90%
#资产合计=长期股权投资+固定资产+无形资产+流动资产	 [1.95809827 1.17303345 1.1051202  1.04137252] b:34492226.29	0.9922	60%
#流动负债=应付账款+预收账款	                                 [1.50860737 1.60131861] b:146408452.47	                                   0.9917  	77%
#负债合计=流动负债+长期负债	                              [1.00009591 0.99979242] b:-36142.11	                                       0.9999	90%
#权益合计=实收资本+资本公积金+盈余公积金	       [2.13877318 1.60430789 4.67835892] b:-72396553.92	              0.8915	94%
#资产合计=负债合计+权益合计                         	[0.99993462 1.00009754] b:-13083	                                        0.9999	96%

In [40]:
import pandas as pd
df= pd.read_excel(r'.././Documents/赛题1数据集/上市信息财务信息-资产负债表(单位消除).xlsx')
A= ['资产:货币资金(元)', '资产:应收账款(元)', '资产:其它应收款(元)', '资产:存货(元)', '资产:流动资产合计(元)',  '资产:长期股权投资(元)', '资产:固定资产(元)', '资产:无形资产(元)', '资产:资产总计(元)', '负债:应付账款(元)', '负债:预收账款(元)',
  '负债:流动负债合计(元)', '负债:长期负债合计(元)','负债:负债合计(元)', '权益:实收资本(或股本)(元)','权益:资本公积金(元)', '权益:盈余公积金(元)',
  '权益:股东权益合计(元)']
df.info()
df=df.drop(columns=['资产:累计折旧(元)','流动比率','负债:存货跌价准备(元)'])
for arr in A:
    df[arr]=pd.to_numeric(df[arr],errors='coerce')
df_asset=df.dropna()
df_asset.info()
#总样本：19620
#完整样本：9298
#比例：50% 根据资产，负债，权益，分别做回归，这样完整样本率也比较高



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 24 columns):
企业编号               19620 non-null int64
标题                 19620 non-null object
日期                 19620 non-null object
资产:货币资金(元)         18862 non-null object
资产:应收账款(元)         18291 non-null object
资产:其它应收款(元)        18460 non-null object
资产:存货(元)           18115 non-null object
资产:流动资产合计(元)       18455 non-null object
资产:长期股权投资(元)       12873 non-null object
资产:累计折旧(元)         1446 non-null object
资产:固定资产(元)         18854 non-null object
资产:无形资产(元)         18595 non-null object
资产:资产总计(元)         18862 non-null object
负债:应付账款(元)         15646 non-null object
负债:预收账款(元)         18117 non-null object
负债:存货跌价准备(元)       1446 non-null object
负债:流动负债合计(元)       18455 non-null object
负债:长期负债合计(元)       17729 non-null object
负债:负债合计(元)         18862 non-null object
权益:实收资本(或股本)(元)    18851 non-null object
权益:资本公积金(元)        18731 non-null object
权益:盈余公积金(元)        18669 non-null

In [44]:
#流动资产相关性分析
data_1= {'货金':df['资产:货币资金(元)'],'应收': df['资产:应收账款(元)'],'其他应收': df['资产:其它应收款(元)'],
         '存货': df['资产:存货(元)'],'流动资产合计':df['资产:流动资产合计(元)']}
data_1= pd.DataFrame(data_1) 
data_1.info()
data_11=data_1.dropna()
data_11.info()
#流动资产类完整样本占比：17739/19620=90%
data_11.corr()
#l流动资产和其他各项特征相关性较大

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 5 columns):
货金        18831 non-null float64
应收        18236 non-null float64
其他应收      18421 non-null float64
存货        18054 non-null float64
流动资产合计    18424 non-null float64
dtypes: float64(5)
memory usage: 766.5 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17739 entries, 6 to 19619
Data columns (total 5 columns):
货金        17739 non-null float64
应收        17739 non-null float64
其他应收      17739 non-null float64
存货        17739 non-null float64
流动资产合计    17739 non-null float64
dtypes: float64(5)
memory usage: 831.5 KB


,货金,应收,其他应收,存货,流动资产合计
货金,1.000000,0.947492,0.922604,0.941661,0.976030
应收,0.947492,1.000000,0.901809,0.925919,0.968374
其他应收,0.922604,0.901809,1.000000,0.987842,0.970685
存货,0.941661,0.925919,0.987842,1.000000,0.985829
流动资产合计,0.976030,0.968374,0.970685,0.985829,1.000000


In [66]:
#流动资产线性回归
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train,X_test,Y_train,Y_test = train_test_split(data_11.iloc[:,:4],data_11['流动资产合计'],train_size=.80)
model = LinearRegression()
model.fit(X_train,Y_train)
a1 = model.intercept_ 
b1 = model.coef_
print("最佳拟合线:截距",a1,",回归系数：",b1)
score = model.score(X_test,Y_test)
print("回归得分:",score)
#回归模型得分较高，建议用流动资产作为其他特征的特征替代

最佳拟合线:截距 23869542.650865078 ,回归系数： [1.48661497 1.06789572 1.35391323 0.94856807]
回归得分: 0.9978899267742676


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [58]:
#资产大类分析
data_3= {'长投':df['资产:长期股权投资(元)'],'固定资产': df['资产:固定资产(元)'],'无形资产': df['资产:无形资产(元)'],
         '流动资产合计':df['资产:流动资产合计(元)'],'总计': df['资产:资产总计(元)'],}
data_3= pd.DataFrame(data_3) 
data_3.info()
data_31=data_3.dropna()
data_31.info()
#资产大类完整样本占比：11884/19620=60%
#回归
X_train,X_test,Y_train,Y_test = train_test_split(data_31.iloc[:,:4],data_31['总计'],train_size=.80)
model = LinearRegression()
model.fit(X_train,Y_train)
a3 = model.intercept_ 
b3 = model.coef_
print("最佳拟合线:截距",a3,",回归系数：",b3)
score = model.score(X_test,Y_test)
print("回归得分:",score)
#高度拟合

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 5 columns):
长投        12342 non-null float64
固定资产      18823 non-null float64
无形资产      18540 non-null float64
流动资产合计    18424 non-null float64
总计        18836 non-null float64
dtypes: float64(5)
memory usage: 766.5 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11884 entries, 6 to 19613
Data columns (total 5 columns):
长投        11884 non-null float64
固定资产      11884 non-null float64
无形资产      11884 non-null float64
流动资产合计    11884 non-null float64
总计        11884 non-null float64
dtypes: float64(5)
memory usage: 557.1 KB
最佳拟合线:截距 34492226.29191351 ,回归系数： [1.95809827 1.17303345 1.1051202  1.04137252]
回归得分: 0.992227005087436


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [59]:
#流动负债回归
data_2 = {'应付':df['负债:应付账款(元)'],
        '预收': df['负债:(元)'],
        '流动负债': df['负债:流动负债合计(元)'],}
data_2= pd.DataFrame(data_2)
data_2.info()
data_21=data_2.dropna()
data_21.info()
#流动负债类完整样本占比：15161/19620=77%
#回归
X_train,X_test,Y_train,Y_test = train_test_split(data_21.iloc[:,:2],data_21['流动负债'],train_size=.80)
model = LinearRegression()
model.fit(X_train,Y_train)
a2= model.intercept_ 
b2= model.coef_
print("最佳拟合线:截距",a2,",回归系数：",b2)
score = model.score(X_test,Y_test)
print("回归得分:",score)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 3 columns):
应付      15595 non-null float64
预收      18068 non-null float64
流动负债    18424 non-null float64
dtypes: float64(3)
memory usage: 459.9 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15161 entries, 7 to 19619
Data columns (total 3 columns):
应付      15161 non-null float64
预收      15161 non-null float64
流动负债    15161 non-null float64
dtypes: float64(3)
memory usage: 473.8 KB
最佳拟合线:截距 146408452.4728253 ,回归系数： [1.50860737 1.60131861]
回归得分: 0.9917740743049736


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [61]:
data_4 = {'流动负债':df['负债:流动负债合计(元)'],
        '长负': df['负债:长期负债合计(元)'],
        '负债合计':df["负债:负债合计(元)"] ,}
data_4= pd.DataFrame(data_4)
data_4.info()
data_41=data_4.dropna()
data_41.info()
#负债大类完整样本占比：17643/19620=90%
#回归
X_train,X_test,Y_train,Y_test = train_test_split(data_41.iloc[:,:2],data_41['负债合计'],train_size=.80)
model = LinearRegression()
model.fit(X_train,Y_train)
a4= model.intercept_ 
b4= model.coef_
print("最佳拟合线:截距",a4,",回归系数：",b4)
score = model.score(X_test,Y_test)
print("回归得分:",score)
#回归效果较好

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 3 columns):
流动负债    18424 non-null float64
长负      17643 non-null float64
负债合计    18831 non-null float64
dtypes: float64(3)
memory usage: 459.9 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17643 entries, 6 to 19619
Data columns (total 3 columns):
流动负债    17643 non-null float64
长负      17643 non-null float64
负债合计    17643 non-null float64
dtypes: float64(3)
memory usage: 551.3 KB
最佳拟合线:截距 -36142.11179125309 ,回归系数： [1.00009591 0.99979242]
回归得分: 0.9999999339252738


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [62]:
#权益回归分析
data_5 = {'实收':df['权益:实收资本(或股本)(元)'],
        '资积': df['权益:资本公积金(元)'],
        '盈积': df['权益:盈余公积金(元)'],'合计':df["权益:股东权益合计(元)"],}
data_5= pd.DataFrame(data_5)
data_5.info()
data_51=data_5.dropna()
data_51.info()
#权益类完整样本占比：18481/19620=94%
X_train,X_test,Y_train,Y_test = train_test_split(data_51.iloc[:,:3],data_51['合计'],train_size=.80)
model = LinearRegression()
model.fit(X_train,Y_train)
a5= model.intercept_ 
b5= model.coef_
print("最佳拟合线:截距",a5,",回归系数：",b5)
score = model.score(X_test,Y_test)
print("回归得分:",score)
#回归效果较好

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 4 columns):
实收    18820 non-null float64
资积    18689 non-null float64
盈积    18610 non-null float64
合计    18823 non-null float64
dtypes: float64(4)
memory usage: 613.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18481 entries, 0 to 19619
Data columns (total 4 columns):
实收    18481 non-null float64
资积    18481 non-null float64
盈积    18481 non-null float64
合计    18481 non-null float64
dtypes: float64(4)
memory usage: 721.9 KB
最佳拟合线:截距 -72396553.92239672 ,回归系数： [2.13877318 1.60430789 4.67835892]
回归得分: 0.8914955247568054


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [63]:
#资产，负债，所有者权益做相关性分析
data = {'资产':df['资产:资产总计(元)'],
        '负债': df['负债:负债合计(元)'],
        '权益': df['权益:股东权益合计(元)'],}
data= pd.DataFrame(data)
data.head(n=5)
data.info()
data1=data.dropna()
data1.info()
#总样本：19620
#完整样本：18823
#比例：95.9%
data1.corr()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 3 columns):
资产    18836 non-null float64
负债    18831 non-null float64
权益    18823 non-null float64
dtypes: float64(3)
memory usage: 459.9 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18823 entries, 0 to 19619
Data columns (total 3 columns):
资产    18823 non-null float64
负债    18823 non-null float64
权益    18823 non-null float64
dtypes: float64(3)
memory usage: 588.2 KB


,资产,负债,权益
资产,1.000000,0.997025,0.960150
负债,0.997025,1.000000,0.935752
权益,0.960150,0.935752,1.000000


In [67]:
#资产负债权益三个特征回归分析
X_train,X_test,Y_train,Y_test = train_test_split(data1.iloc[:,1:3],data1['资产'],train_size=.80)
model = LinearRegression()
model.fit(X_train,Y_train)
a  = model.intercept_ 
b = model.coef_
print("最佳拟合线:截距",a,",回归系数：",b)
#资产=负债+权益-38545。
score = model.score(X_test,Y_test)
print("回归得分:",score)
#本来打算通过回归可以计算缺失值，但是发现这三者基本是同时缺失的，放弃此方法

最佳拟合线:截距 -13083.008670687675 ,回归系数： [0.99993462 1.00009754]
回归得分: 0.9999999227067733


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
